# Tracing 101

Step through this notebook to understand how tracing works in Generative AI Toolkit.

The Generative AI Toolkit comes with these tracers out-of-the-box:


In [ ]:
from generative_ai_toolkit.tracer import (
    NoopTracer,
    HumanReadableTracer,
    InMemoryTracer,
    StructuredLogsTracer,
    TeeTracer,
)
from generative_ai_toolkit.tracer.dynamodb import DynamoDbTracer

import time
import random

### `InMemoryTracer`

Use the in-memory tracer for testing and development:


In [ ]:
in_memory_tracer = InMemoryTracer(
    memory_size=1000  # Store max 1000 traces, before discarding older ones
)
in_memory_tracer.set_context(
    resource_attributes={"service.name": "MyAgent"}
)  # Context, added to all traces
with in_memory_tracer.trace("parent") as parent_span:
    parent_span.add_attribute("foo", "bar")
    parent_span.add_attribute(
        "inherited.foo", "bar", inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with in_memory_tracer.trace("parent") as child_span:
        child_span.add_attribute("bar", "foo")
        time.sleep(0.1)
for trace in in_memory_tracer.get_traces():
    print(trace)
    print()

### Printing a human-readable version of traces during development

In the following example we add attributes that Generative AI Toolkit understands. It will use these to present traces in a way that is nicer to the human eye:


In [ ]:
conversation_id = random.randint(0, 1000000)

with in_memory_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute(
        "ai.conversation.id", conversation_id, inheritable=True
    )  # Inheritable attributes propagate to child spans
    parent_span.add_attribute(
        "ai.auth.context", "user123", inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with in_memory_tracer.trace("parent") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in in_memory_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id
    }  # filter traces by conversation id
):
    print(trace.as_human_readable())
    print()

### `HumanReadableTracer`

You can also use the `HumanReadableTracer` that will log traces in human readable form to stdout, which is useful during development.

Note that traces are logged when the span ends, so parent spans are logged after child spans (this is true for all tracers):


In [ ]:
import sys

human_readable_tracer = HumanReadableTracer(stream=sys.stdout)
human_readable_tracer.set_context(
    resource_attributes={"service.name": "MyAgent"}
)  # Context, added to all traces
with human_readable_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute(
        "ai.conversation.id", conversation_id, inheritable=True
    )  # Inheritable attributes propagate to child spans
    parent_span.add_attribute(
        "ai.auth.context", "user123", inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with human_readable_tracer.trace("parent") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

### `StructuredLogsTracer`

Use the `StructuredLogsTracer` to log traces to stdout as JSON:


In [ ]:
structured_logs_tracer = StructuredLogsTracer(stream=sys.stdout)
structured_logs_tracer.set_context(
    resource_attributes={"service.name": "MyAgent"}
)  # Context, added to all traces
with structured_logs_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute(
        "ai.conversation.id", conversation_id, inheritable=True
    )  # Inheritable attributes propagate to child spans
    parent_span.add_attribute(
        "ai.auth.context", "user123", inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with structured_logs_tracer.trace("parent") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

### `DynamoDbTracer`

Use the `DynamoDbTracer` to store traces to DynamoDB.

To use this tracer, you should have created a table with partition key `pk` (string) and sort key `sk` (string).

If you want to support getting traces by conversation ID, the table must have a GSI with partition key `conversation_id` (string) and sort key `sk` (string).

For example, here's how to create such a table:


In [ ]:
!aws dynamodb create-table \
  --table-name MyTracesTable \
  --attribute-definitions \
    AttributeName=pk,AttributeType=S \
    AttributeName=sk,AttributeType=S \
    AttributeName=conversation_id,AttributeType=S \
  --key-schema \
    AttributeName=pk,KeyType=HASH \
    AttributeName=sk,KeyType=RANGE \
  --billing-mode PAY_PER_REQUEST \
  --global-secondary-indexes '[{"IndexName":"conversation_index","KeySchema":[{"AttributeName":"conversation_id","KeyType":"HASH"},{"AttributeName":"sk","KeyType":"RANGE"}],"Projection":{"ProjectionType":"ALL"}}]'

Then, use that table in the `DynamoDbTracer`:


In [ ]:
conversation_id = random.randint(0, 1000000)
auth_context = "user123"

ddb_tracer = DynamoDbTracer(
    table_name="MyTracesTable",
    identifier="MyAgent",
    conversation_id_gsi_name="conversation_index",
)
ddb_tracer.set_context(
    resource_attributes={"service.name": "MyAgent"}
)  # Context, added to all traces
with ddb_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute(
        "ai.conversation.id", conversation_id, inheritable=True
    )  # Inheritable attributes propagate to child spans
    parent_span.add_attribute(
        "ai.auth.context", auth_context, inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with ddb_tracer.trace("parent") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in ddb_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace.as_human_readable())
    print()

### `NoopTracer`

Use the no-operation tracer when you don't want traces:


In [ ]:
noop_tracer = NoopTracer()
with noop_tracer.trace("noop") as span:
    span.add_attribute("foo", "bar")

# nothing was logged

### `TeeTracer`

Use the `TeeTracer` to send traces to multiple tracers at once.

Note that the first tracer you add, will be the one that `get_traces()` will be delegated to. So if you want to use that method, use a tracer that supports it.

Add tracers like this:


In [ ]:
tee_tracer = TeeTracer()

tee_tracer.add_tracer(
    ddb_tracer
)  # The DynamoDBTracer supports get_traces(), so added first
tee_tracer.add_tracer(human_readable_tracer)
tee_tracer.add_tracer(
    noop_tracer
)  # This is of course useless, added for the sake of the example

Then, use the `TeeTracer` as any other tracer:


In [ ]:
conversation_id = random.randint(0, 1000000)
auth_context = "user456"

tee_tracer.set_context(
    resource_attributes={"service.name": "MyAgent"}
)  # Context, added to all traces

with tee_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute(
        "ai.conversation.id", conversation_id, inheritable=True
    )  # Inheritable attributes propagate to child spans
    parent_span.add_attribute(
        "ai.auth.context", auth_context, inheritable=True
    )  # Inheritable attributes propagate to child spans
    time.sleep(0.1)
    with tee_tracer.trace("parent") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


print("==== from DynamoDB: ====")
for trace in tee_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace)
    print()